In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 75%; }
</style>
"""))

In [2]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is 686D-8B85

 Directory of C:\Users\handy89\deltax\RCSnail-AI-lite\src

01/09/2021  06:37 PM    <DIR>          .
01/09/2021  06:37 PM    <DIR>          ..
01/09/2021  03:25 PM    <DIR>          .ipynb_checkpoints
01/09/2021  01:34 PM             3,620 clean_json_asRCSnail_masked.py
01/09/2021  01:55 PM            17,538 example_input_in_try_loop.png
12/22/2020  06:03 PM    <DIR>          learning
12/22/2020  03:19 PM             3,051 main.py
12/23/2020  12:49 PM             6,679 main_old.py
12/22/2020  03:19 PM             6,164 main_ubuntu.py
01/09/2021  01:54 PM             7,171 main_windows.py
12/22/2020  05:52 PM         5,864,112 new_0.13_0.21.h5
01/09/2021  06:15 PM    <DIR>          preprocessed
01/09/2021  01:55 PM            23,499 raw_input_in_try_loop.png
01/09/2021  06:37 PM            40,481 training_ground-handy.ipynb
12/22/2020  06:03 PM    <DIR>          utilities
               9 File(s)      5,972,315 bytes
     

In [3]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
#from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
#from src.learning.models import create_standalone_nvidia_cnn, create_standalone_resnet
                                

In [4]:
def plot_stuff(title, plot_elem, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    #x = np.arange(0, len(plot_elems[0]['data']), 1)
    
    #for plot_elem in plot_elems:
    #    plt.errorbar(x, plot_elem['data'], yerr=plot_elem['error'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o', capsize=5)
    
    plt.plot(list(range(1,len(plot_elem['data'])+1)), plot_elem['data'])
    plt.grid(axis='both')
    #plt.legend(loc='best', prop={'size': 15})
    plt.show()
    plt.savefig('./' + title + '.png')
    
def get_model_num(model_path, model_prefix):
    model_files = [fn for fn in os.listdir(model_path) if fn.startswith(model_prefix) and fn.endswith('.h5') and 'dagger_' not in fn]
    # expected format is "model_n1_m1_9.h5"
    existing_nums = [int(fn.split('_')[3].split('.')[0]) for fn in model_files]
    
    if len(existing_nums) == 0:
        return 1
    else:
        latest_num = sorted(existing_nums)[-1]
        return int(latest_num) + 1

In [5]:
config_manager = ConfigurationManager()
config = config_manager.config

memory = (1, 1)
#ARDI' comment -- we just use (1,1) this refers to how many frames are used and sth else, not sure what

model_path = '../'

In [6]:
import tensorflow as tf
def scheduler(epoch, lr):
    if epoch>30 and epoch%10==0:
        lr*=0.5
    return lr


# Model experiments
import os

epochs = 50
batch_size = 64
verbose = 1

losses = []
val_losses = []

#in here added option to not shuffle, so last 20% of recording time is used as val set -- in future might want to reduce proportion of val set
#generator = Generator(config, memory_tuple= memory, base_path='../', batch_size=batch_size, column_mode='all', shuffle_data=False) 

#frame_shape, numeric_shape = generator.get_shapes()
#tqdm.write('Target shape: {}'.format(diff_shape)) #tqdm is some package that allow to track the progress of operations
#tqdm.write('Input shapes: {}; {}'.format(frame_shape, numeric_shape))

models = []

#ARDI's comment:  Nividia is the model we want to use (resnet might be good to??)
#generator = Generator(config, memory_tuple= memory, base_path='../', batch_size=batch_size, column_mode='all', shuffle_data=False) 
models.append((create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=2), generator.generate))

#"steer and throttle"
#models.append((create_small_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=2), generator.generate))

#generator = Generator(config, memory_tuple= memory, base_path='../', batch_size=batch_size, column_mode='all', shuffle_data=False) 
#models.append((create_small_cnn(activation='linear', input_shape=(50, 180, 3), output_shape=2), generator.generate))

callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)]

for model, generate_method in tqdm(models):
    result_desc = 'n{}_m{}'.format(*memory)
    tqdm.write(result_desc)

    hist = model.fit(generate_method(data='train'),
                     steps_per_epoch=generator.train_batch_count,
                     validation_data=generate_method(data='test'),
                     validation_steps=generator.test_batch_count,
                     callbacks=callbacks,
                     epochs=epochs, verbose=verbose)

    model_file_prefix = 'model_n{}_m{}'.format(*memory)
    model_file_suffix = '_{}.{}'

    model_number = get_model_num(model_path, model_file_prefix)
    plot_model(model, to_file=model_path + model_file_prefix + model_file_suffix.format(model_number, 'png'), show_shapes=True)
    model.save(model_path + model_file_prefix + model_file_suffix.format(model_number, 'h5'))
    
    current_loss = hist.history['loss']
    current_val_loss = hist.history['val_loss'] 
    
    losses.append(current_loss)
    print(val_losses)
    val_losses.append(current_val_loss)
    
    tqdm.write("Loss per epoch: {}".format(current_loss))
    tqdm.write("Validation loss per epoch: {}".format(current_val_loss))
    
    gc.collect()

os.system("printf '\a'")

NameError: name 'create_standalone_nvidia_cnn' is not defined

In [ ]:
print(val_losses)
print(hist.history['val_loss'] )
loss_data = []
val_loss_data = []


val_loss_data = {'data': hist.history['val_loss'], 'label': 'Validation loss', 'alpha': 1.0}
plot_stuff('Nivida cnn standalone validation loss', val_loss_data, figsize=(10, 6))


In [ ]:
loss_data = {'data': losses[0], 'label': 'Loss', 'alpha': 1.0}
plot_stuff('Training', loss_data, figsize=(10, 6))

In [ ]:
#todo load test set into memeory, evaluate
import keras
model = keras.models.load_model("../masked_test.h5")
import glob

val_data_loc="../n1_m1"
filenames = glob.glob("../n1_m1/*")
print(len(filenames))
nr_of_datapoints = int(len(filenames)/2) #label and image files


MAEs=[]
preds=[]
labels=[]

for batch in range(nr_of_datapoints//32-32,nr_of_datapoints//32): # using the end of file. 32 batches of size batch of 32
    frames=np.zeros((32,50,180,3))
    commands = np.zeros((32,2))
    for i in range(32):
        frames[i,:] = np.load("../n1_m1/frame_"+str(batch*32+i).zfill(7)+".npy")
        commands[i] = np.load("../n1_m1/commands_"+str(batch*32+i).zfill(7)+".npy")
    MAEs.append(model.evaluate(frames,commands, batch_size=32))
    pred = model.predict(frames)
    preds.append(pred)
    labels.append(commands)
    
print(np.mean(MAEs))
#TODO MAE for steer and throttle separately, mased on preds and commands

In [ ]:
print(np.mean(MAEs))
p_steer=np.array(preds)[:,:,0]
l_steer=np.array(labels)[:,:,0]
p_throttle=np.array(preds)[:,:,1]
l_throttle=np.array(labels)[:,:,1]

print(p_steer.shape,l_steer.shape)
p_steer=p_steer.flatten()
l_steer=l_steer.flatten()
p_throttle=p_throttle.flatten()
l_throttle=l_throttle.flatten()


plt.figure(figsize=(32,8))
plt.plot(range(1000),l_steer[:1000],label="human steering")
plt.plot(range(1000),p_steer[:1000],label="predicted")
plt.legend()
plt.show()


plt.figure(figsize=(32,8))
plt.plot(range(1000),l_throttle[:1000],label="human throttle")
plt.plot(range(1000),p_throttle[:1000],label="predicted")
plt.legend()
plt.show()



plt.scatter(l_steer,p_steer,s=0.001)

# Handy Testing

In [13]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import skimage.io as io
import skimage
from skimage.transform import rescale, resize, downscale_local_mean

import glob
import random
from PIL import Image
import PIL
 

In [8]:
import glob

val_data_loc="preprocessed/cleaned_all"
filenames = glob.glob("preprocessed/cleaned_all/*")
print(len(filenames))
nr_of_datapoints = int(len(filenames)/340) #label and image files
print(nr_of_datapoints)


MAEs=[]
preds=[]
labels=[]


10384
30


In [36]:
frames = []
commands = []
for batch in range(1,nr_of_datapoints): # using the end of file. 32 batches of size batch of 32
    #frames=np.zeros((1,60,180,3))
    #commands = np.zeros((1,2))
    
    frames.append(np.load("preprocessed/cleaned_all/frame_"+str(batch).zfill(7)+".npy"))
    commands.append(np.load("preprocessed/cleaned_all/commands_"+str(batch).zfill(7)+".npy"))
    
    
    #print(commands)
    
    
    
    #MAEs.append(model.evaluate(frames,commands, batch_size=32))
    #pred = model.predict(frames)
    #preds.append(pred)
    #labels.append(commands)

In [39]:
#frames = frames.reshape(1,60,180,3)
frames = np.array(frames)
frames.shape

(29, 60, 180, 3)

In [40]:
#commands = commands.reshape(1, 2)
commands = np.array(commands)
commands.shape

(29, 2)

In [15]:
model = create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=2)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 60, 180, 3)]      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 90, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 45, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 23, 48)         43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 23, 64)         27712     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 23, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 11776)             0   

In [45]:
model.fit(frames, commands, batch_size=25, epochs=2, validation_split=0.2)

Train on 23 samples, validate on 6 samples
Epoch 1/2
23/23 [==============================] - 0s 174us/sample


TypeError: 'NoneType' object is not callable

In [7]:
def create_standalone_nvidia_cnn(activation='linear', input_shape=(60, 180, 3), output_shape=1):
    """
    Activation: linear, softmax.
    Architecture is from nvidia paper mentioned in https://github.com/tanelp/self-driving-convnet/blob/master/train.py
    """
    from tensorflow.keras.layers import Convolution2D
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Flatten
    from tensorflow.keras.layers import Input
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.losses import mean_squared_error, mean_absolute_error

    inputs = Input(shape=input_shape)
    conv_1 = Convolution2D(24, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(inputs)
    conv_2 = Convolution2D(36, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(conv_1)
    conv_3 = Convolution2D(48, kernel_size=(5, 5), kernel_regularizer=l2(0.0005), strides=(2, 2), padding="same", activation="elu")(conv_2)
    conv_4 = Convolution2D(64, kernel_size=(3, 3), kernel_regularizer=l2(0.0005), padding="same", activation="elu")(conv_3)
    conv_5 = Convolution2D(64, kernel_size=(3, 3), kernel_regularizer=l2(0.0005), padding="same", activation="elu")(conv_4)
    flatten = Flatten()(conv_5)
    dense_1 = Dense(1164, kernel_regularizer=l2(0.0005), activation="elu")(flatten)
    dense_2 = Dense(100, kernel_regularizer=l2(0.0005), activation="elu")(dense_1)
    dense_3 = Dense(50, kernel_regularizer=l2(0.0005), activation="elu")(dense_2)
    dense_4 = Dense(10, kernel_regularizer=l2(0.0005), activation="elu")(dense_3)
    out_dense = Dense(output_shape, activation=activation)(dense_4)

    model = Model(inputs=inputs, outputs=out_dense)
    optimizer = Adam(lr=3e-4)
    model.compile(loss=mean_absolute_error, optimizer=optimizer)

    return model